# Advent of Code

## 2018-012-017
## 2018 017

https://adventofcode.com/2018/day/17

In [5]:
import re

import numpy as np



# Parse the input

input_path = 'input.txt'
def parse_input_fixed(input_path):

    clay_positions = []

    with open(input_path, 'r') as file:

        for line in file:

            line = line.strip()

            if line.startswith("x"):

                x, y1, y2 = map(int, re.findall(r"x=(\d+), y=(\d+)\.\.(\d+)", line)[0])

                clay_positions.extend((x, y) for y in range(y1, y2 + 1))

            elif line.startswith("y"):

                y, x1, x2 = map(int, re.findall(r"y=(\d+), x=(\d+)\.\.(\d+)", line)[0])

                clay_positions.extend((x, y) for x in range(x1, x2 + 1))

    return clay_positions



# Retry parsing with corrected logic

clay_positions = parse_input_fixed(input_path)



# Define grid boundaries

min_x = min(x for x, y in clay_positions)

max_x = max(x for x, y in clay_positions)

min_y = min(y for x, y in clay_positions)

max_y = max(y for x, y in clay_positions)



# Extend the grid for overflow

padding = 1

grid_width = max_x - min_x + 1 + 2 * padding

grid_height = max_y + 1



# Create the grid

grid = np.full((grid_height, grid_width), '.', dtype=str)



# Place clay on the grid

for x, y in clay_positions:

    grid[y, x - min_x + padding] = '#'



# Place the water spring

spring_x = 500 - min_x + padding

grid[0, spring_x] = '+'
def simulate_water(grid, spring_x, min_y, max_y):
    flowing_water = set()
    resting_water = set()
    
    def flow_down(x, y):
        while y < max_y and grid[y, x] == '.':
            grid[y, x] = '|'
            flowing_water.add((x, y))
            y += 1
        if y >= max_y:
            return
        if grid[y, x] in ['#', '~']:
            y -= 1
            flow_out(x, y)

    def flow_out(x, y):
        left_bound = x
        while left_bound > 0 and grid[y, left_bound] == '.' and grid[y + 1, left_bound] in ['#', '~']:
            grid[y, left_bound] = '|'
            flowing_water.add((left_bound, y))
            left_bound -= 1
        right_bound = x
        while right_bound < grid.shape[1] - 1 and grid[y, right_bound] == '.' and grid[y + 1, right_bound] in ['#', '~']:
            grid[y, right_bound] = '|'
            flowing_water.add((right_bound, y))
            right_bound += 1

        if grid[y + 1, left_bound] not in ['#', '~']:
            flow_down(left_bound, y + 1)
        if grid[y + 1, right_bound] not in ['#', '~']:
            flow_down(right_bound, y + 1)

        if grid[y, left_bound] == '#' and grid[y, right_bound] == '#':
            for i in range(left_bound + 1, right_bound):
                grid[y, i] = '~'
                resting_water.add((i, y))

    flow_down(spring_x, 1)

    return flowing_water, resting_water

# Retry the simulation
flowing_water, resting_water = simulate_water(grid, spring_x, min_y, max_y)

# Count reachable tiles
reachable_tiles = len(flowing_water | resting_water)

reachable_tiles

23

In [1]:
import re
from collections import deque

# Read the input from input.txt
with open('input.txt', 'r') as file:
    input_data = file.read()

clay_coords = set()
min_x = min_y = float('inf')
max_x = max_y = float('-inf')

# Parse each line of the input
for line in input_data.strip().split('\n'):
    nums = list(map(int, re.findall(r'\d+', line)))
    if line.startswith('x'):
        x = nums[0]
        y1, y2 = nums[1], nums[2]
        for y in range(y1, y2 + 1):
            clay_coords.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    else:
        y = nums[0]
        x1, x2 = nums[1], nums[2]
        for x in range(x1, x2 + 1):
            clay_coords.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)

# Expand the grid boundaries a bit to account for water spreading
min_x -= 1
max_x += 1

# Initialize the grid
grid = {}
for y in range(0, max_y + 2):
    for x in range(min_x, max_x + 1):
        grid[(x, y)] = '.'

# Place the clay
for (x, y) in clay_coords:
    grid[(x, y)] = '#'

# Place the water spring
spring_x, spring_y = 500, 0
grid[(spring_x, spring_y)] = '+'

# Simulate water flow
def flow(x, y, from_above=False):
    if y > max_y:
        return
    if grid[(x, y)] in ('#', '~'):
        return
    if grid[(x, y)] == '|':
        if from_above:
            return
    grid[(x, y)] = '|'
    below = grid.get((x, y + 1), '.')
    if below == '.':
        flow(x, y + 1)
    elif below in ('#', '~'):
        # Try to spread left and right
        left_bound = x
        right_bound = x

        # Spread left
        while True:
            left = left_bound - 1
            below_left = grid.get((left, y + 1), '.')
            if grid.get((left, y), '.') in ('#', '~'):
                break
            if below_left not in ('#', '~'):
                flow(left, y)
                break
            left_bound -= 1
            grid[(left, y)] = '|'

        # Spread right
        while True:
            right = right_bound + 1
            below_right = grid.get((right, y + 1), '.')
            if grid.get((right, y), '.') in ('#', '~'):
                break
            if below_right not in ('#', '~'):
                flow(right, y)
                break
            right_bound += 1
            grid[(right, y)] = '|'

        # Check if water is settled
        is_settled = True
        for x_fill in range(left_bound, right_bound + 1):
            if grid.get((x_fill, y + 1), '.') not in ('#', '~'):
                is_settled = False
                break
        if is_settled:
            for x_fill in range(left_bound, right_bound + 1):
                grid[(x_fill, y)] = '~'
            flow(x, y - 1, from_above=True)
    else:
        # Flowing into abyss
        return

# Start the water flow
flow(spring_x, spring_y + 1)

# Count the water tiles
water_tiles = 0
for (x, y), value in grid.items():
    if min_y <= y <= max_y and value in ('|', '~'):
        water_tiles += 1

print(f"Total number of tiles the water can reach: {water_tiles}")

Total number of tiles the water can reach: 38


In [13]:
import re
from collections import deque

def parse_input(input_lines):
    clay = set()
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line: x=495, y=2..7 or y=7, x=495..501
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_y, max_y

def simulate_water(clay, min_y, max_y):
    water = set()
    settled = set()
    stack = deque()
    stack.append((500, 0))

    while stack:
        x, y = stack.pop()
        if y > max_y:
            continue
        if (x, y) in water:
            continue
        water.add((x, y))
        # Move down
        if (x, y + 1) not in clay and (x, y + 1) not in settled:
            stack.append((x, y + 1))
            continue
        # Now, try to spread left and right
        left = x
        right = x
        blocked_left = False
        blocked_right = False

        # Spread left
        while True:
            left -= 1
            if (left, y) in clay:
                blocked_left = True
                break
            if (left, y + 1) not in clay and (left, y + 1) not in settled:
                stack.append((left, y))
                break
            water.add((left, y))
        # Spread right
        while True:
            right += 1
            if (right, y) in clay:
                blocked_right = True
                break
            if (right, y + 1) not in clay and (right, y + 1) not in settled:
                stack.append((right, y))
                break
            water.add((right, y))
        # If both sides are blocked, water settles
        if blocked_left and blocked_right:
            for settle_x in range(left + 1, right):
                settled.add((settle_x, y))
        # Else, water remains flowing

    # Combine flowing and settled water
    all_water = water.union(settled)
    # Only count water within min_y and max_y
    reachable = set()
    for (x, y) in all_water:
        if min_y <= y <= max_y:
            reachable.add((x, y))
    return reachable

def main():
    # Example input
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    clay, min_y, max_y = parse_input(input_data)
    reachable = simulate_water(clay, min_y, max_y)
    print(f"Number of tiles the water can reach: {len(reachable)}")

if __name__ == "__main__":
    main()

Number of tiles the water can reach: 10


In [14]:
import re
from collections import deque

def parse_input(input_lines):
    clay = set()
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line: x=495, y=2..7 or y=7, x=495..501
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_y, max_y

def simulate_water(clay, min_y, max_y):
    water = set()
    settled = set()
    stack = deque()
    stack.append((500, 0))

    while stack:
        x, y = stack.pop()
        if y > max_y:
            continue
        if (x, y) in water or (x, y) in settled:
            continue
        water.add((x, y))
        # Move down
        if (x, y + 1) not in clay and (x, y + 1) not in settled:
            stack.append((x, y + 1))
            continue
        # Now, try to spread left and right
        left = x
        right = x
        blocked_left = False
        blocked_right = False

        # Spread left
        while True:
            left -= 1
            if (left, y) in clay:
                blocked_left = True
                break
            if (left, y + 1) not in clay and (left, y + 1) not in settled:
                stack.append((left, y))
                break
            water.add((left, y))
        # Spread right
        while True:
            right += 1
            if (right, y) in clay:
                blocked_right = True
                break
            if (right, y + 1) not in clay and (right, y + 1) not in settled:
                stack.append((right, y))
                break
            water.add((right, y))
        # If both sides are blocked, water settles
        if blocked_left and blocked_right:
            for settle_x in range(left + 1, right):
                settled.add((settle_x, y))
            # After settling, revisit the row above
            stack.append((x, y - 1))
    # Combine flowing and settled water
    all_water = water.union(settled)
    # Only count water within min_y and max_y
    reachable = set()
    for (x, y) in all_water:
        if min_y <= y <= max_y:
            reachable.add((x, y))
    return reachable

def main():
    # Example input
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    clay, min_y, max_y = parse_input(input_data)
    reachable = simulate_water(clay, min_y, max_y)
    print(f"Number of tiles the water can reach: {len(reachable)}")

if __name__ == "__main__":
    main()

Number of tiles the water can reach: 10


In [15]:
import re
from collections import deque

def parse_input(input_lines):
    clay = set()
    min_y = float('inf')
    max_y = float('-inf')
    for line in input_lines:
        # Parse each line to get fixed and variable axes
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if m:
            axis1, val1, axis2, start, end = m.groups()
            val1 = int(val1)
            start = int(start)
            end = int(end)
            for val2 in range(start, end + 1):
                if axis1 == 'x':
                    x = val1
                    y = val2
                else:
                    x = val2
                    y = val1
                clay.add((x, y))
                # Update min_y and max_y based on clay positions
                if y < min_y:
                    min_y = y
                if y > max_y:
                    max_y = y
    return clay, min_y, max_y

def simulate_water(clay, min_y, max_y):
    water = set()
    settled = set()
    stack = deque()
    stack.append((500, 0))

    while stack:
        x, y = stack.pop()
        if y > max_y:
            continue
        if (x, y) in water or (x, y) in settled or (x, y) in clay:
            continue
        water.add((x, y))
        # Move down
        if (x, y + 1) not in clay and (x, y + 1) not in settled:
            stack.append((x, y + 1))
            continue
        # Now, try to spread left and right
        left = x
        right = x
        blocked_left = False
        blocked_right = False
        row = []
        row.append((x, y))
        # Spread left
        while True:
            left -= 1
            pos = (left, y)
            below = (left, y + 1)
            if pos in clay:
                blocked_left = True
                break
            if below not in clay and below not in settled:
                stack.append(pos)
                break
            row.append(pos)
        # Spread right
        while True:
            right += 1
            pos = (right, y)
            below = (right, y + 1)
            if pos in clay:
                blocked_right = True
                break
            if below not in clay and below not in settled:
                stack.append(pos)
                break
            row.append(pos)
        # If both sides are blocked, water settles
        if blocked_left and blocked_right:
            for pos in row:
                settled.add(pos)
            # After settling, need to check row above
            stack.append((x, y - 1))
        else:
            # Water is still flowing
            for pos in row:
                water.add(pos)
    # Now, count the tiles in water and settled within min_y and max_y
    reachable = set()
    for (x, y) in water.union(settled):
        if min_y <= y <= max_y:
            reachable.add((x, y))
    return len(reachable)

def main():
    # Example input
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    clay, min_y, max_y = parse_input(input_data)
    reachable_tiles = simulate_water(clay, min_y, max_y)
    print(f"Number of tiles the water can reach: {reachable_tiles}")

if __name__ == "__main__":
    main()

Number of tiles the water can reach: 10


In [16]:
import re
from collections import deque

def parse_input(input_lines):
    clay = set()
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line: x=495, y=2..7 or y=7, x=495..501
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_x, max_x, min_y, max_y

def visualize(grid, min_x, max_x, min_y, max_y):
    for y in range(min_y, max_y + 1):
        line = ""
        for x in range(min_x - 1, max_x + 2):
            if (x, y) == (500, 0):
                line += '+'
            elif (x, y) in grid:
                line += grid[(x, y)]
            else:
                line += '.'
        print(f"{y:4} {line}")

def simulate_water(clay, min_x, max_x, min_y, max_y):
    grid = {}
    for (x, y) in clay:
        grid[(x, y)] = '#'

    stack = deque()
    stack.append((500, 0))

    while stack:
        x, y = stack.pop()
        if y > max_y:
            continue
        if (x, y) not in grid:
            grid[(x, y)] = '|'
        # Move down
        if y < max_y:
            below = (x, y + 1)
            if below not in grid:
                stack.append((x, y + 1))
                continue
            if grid.get(below) in ['|']:
                continue
            if grid.get(below) in ['.', '|']:
                stack.append((x, y + 1))
                continue
        # Now, try to spread left and right
        left = x
        right = x

        # Spread left
        while True:
            left -= 1
            current = (left, y)
            below = (left, y + 1)
            if current in grid and grid[current] == '#':
                left += 1
                break
            if below not in grid or grid.get(below) not in ['#', '~']:
                # Water can fall down
                stack.append((left, y))
                break
            grid[current] = '|'

        # Spread right
        while True:
            right += 1
            current = (right, y)
            below = (right, y + 1)
            if current in grid and grid[current] == '#':
                right -= 1
                break
            if below not in grid or grid.get(below) not in ['#', '~']:
                # Water can fall down
                stack.append((right, y))
                break
            grid[current] = '|'

        # Check if water is bounded on both sides
        bounded = True
        left_bound = (left, y) in clay
        right_bound = (right, y) in clay
        if left_bound and right_bound:
            for fill_x in range(left, right + 1):
                grid[(fill_x, y)] = '~'
            # After settling, revisit the row above
            stack.append((x, y - 1))
        else:
            # If not bounded, mark as flowing
            for fill_x in range(left, right + 1):
                if grid.get((fill_x, y)) != '#':
                    grid[(fill_x, y)] = '|'

    return grid

def count_reachable_tiles(grid, min_y, max_y):
    count = 0
    for (x, y), val in grid.items():
        if min_y <= y <= max_y and val in ['|', '~']:
            count += 1
    return count

def main():
    # Example input
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    clay, min_x, max_x, min_y, max_y = parse_input(input_data)
    grid = simulate_water(clay, min_x, max_x, min_y, max_y)
    reachable = count_reachable_tiles(grid, min_y, max_y)
    print(f"Number of tiles the water can reach: {reachable}\n")
    visualize(grid, min_x, max_x, min_y, max_y)

if __name__ == "__main__":
    main()

Number of tiles the water can reach: 10

   1 ......|.....#.
   2 .#..#.|.....#.
   3 .#..#.|#......
   4 .#..#.|#......
   5 .#....|#......
   6 .#|||||#......
   7 .#######......
   8 ..............
   9 ..............
  10 ....#.....#...
  11 ....#.....#...
  12 ....#.....#...
  13 ....#######...


In [6]:
import re
from collections import deque

def parse_input(input_lines):
    clay = set()
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line: x=495, y=2..7 or y=7, x=495..501
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_x, max_x, min_y, max_y

def visualize(grid, min_x, max_x, min_y, max_y):
    for y in range(min_y, max_y + 1):
        line = ""
        for x in range(min_x - 1, max_x + 2):
            if (x, y) == (500, 0):
                line += '+'
            elif (x, y) in grid:
                line += grid[(x, y)]
            else:
                line += '.'
        print(f"{y:4} {line}")

def simulate_water(clay, min_x, max_x, min_y, max_y):
    grid = {}
    for (x, y) in clay:
        grid[(x, y)] = '#'

    stack = deque()
    stack.append((500, 0))

    while stack:
        x, y = stack.pop()
        if y > max_y:
            continue
        if (x, y) not in grid:
            grid[(x, y)] = '|'
        # Move down
        if y < max_y:
            below = (x, y + 1)
            if below not in grid:
                stack.append((x, y + 1))
                continue
            if grid.get(below) in ['|']:
                continue
            if grid.get(below) in ['.', '|']:
                stack.append((x, y + 1))
                continue
        # Now, try to spread left and right
        left = x
        right = x

        # Spread left
        while True:
            left -= 1
            current = (left, y)
            below = (left, y + 1)
            if current in grid and grid[current] == '#':
                left += 1
                break
            if below not in grid or grid.get(below) not in ['#', '~']:
                # Water can fall down
                stack.append((left, y))
                break
            grid[current] = '|'

        # Spread right
        while True:
            right += 1
            current = (right, y)
            below = (right, y + 1)
            if current in grid and grid[current] == '#':
                right -= 1
                break
            if below not in grid or grid.get(below) not in ['#', '~']:
                # Water can fall down
                stack.append((right, y))
                break
            grid[current] = '|'

        # Check if water is bounded on both sides
        bounded = True
        left_bound = (left, y) in clay
        right_bound = (right, y) in clay
        if left_bound and right_bound:
            for fill_x in range(left, right + 1):
                grid[(fill_x, y)] = '~'
            # After settling, revisit the row above
            stack.append((x, y - 1))
        else:
            # If not bounded, mark as flowing
            for fill_x in range(left, right + 1):
                if grid.get((fill_x, y)) != '#':
                    grid[(fill_x, y)] = '|'

    return grid

def count_reachable_tiles(grid, min_y, max_y):
    count = 0
    for (x, y), val in grid.items():
        if min_y <= y <= max_y and val in ['|', '~']:
            count += 1
    return count

def main():
    # Example input
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    clay, min_x, max_x, min_y, max_y = parse_input(input_data)
    grid = simulate_water(clay, min_x, max_x, min_y, max_y)
    reachable = count_reachable_tiles(grid, min_y, max_y)
    print(f"Number of tiles the water can reach: {reachable}\n")
    visualize(grid, min_x, max_x, min_y, max_y)

if __name__ == "__main__":
    main()

Number of tiles the water can reach: 10

   1 ......|.....#.
   2 .#..#.|.....#.
   3 .#..#.|#......
   4 .#..#.|#......
   5 .#....|#......
   6 .#|||||#......
   7 .#######......
   8 ..............
   9 ..............
  10 ....#.....#...
  11 ....#.....#...
  12 ....#.....#...
  13 ....#######...


In [1]:
import re

def parse_input(input_lines):
    """
    Parses the input lines to extract clay vein positions.

    Args:
        input_lines (list of str): Each string represents a clay vein definition.

    Returns:
        tuple: A tuple containing:
            - clay (set of tuples): Set of (x, y) positions occupied by clay.
            - min_x (int): Minimum x-coordinate in the clay veins.
            - max_x (int): Maximum x-coordinate in the clay veins.
            - min_y (int): Minimum y-coordinate in the clay veins.
            - max_y (int): Maximum y-coordinate in the clay veins.
    """
    clay = set()
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line formats:
        # "x=495, y=2..7"
        # "y=7, x=495..501"
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue  # Skip lines that don't match the expected format
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_x, max_x, min_y, max_y

def print_x_labels(min_x, max_x):
    """
    Prints the x-axis labels above the grid for reference.

    Args:
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
    """
    # Create the three label lines: hundreds, tens, units
    hundreds = ""
    tens = ""
    units = ""
    for x in range(min_x, max_x + 1):
        hundreds += str(x // 100)
    for x in range(min_x, max_x + 1):
        tens += str((x % 100) // 10)
    for x in range(min_x, max_x + 1):
        units += str(x % 10)
    print("   " + hundreds)
    print("   " + tens)
    print("   " + units)

def visualize(clay, min_x, max_x, min_y, max_y):
    """
    Renders the grid with clay, sand, and the water spring.

    Args:
        clay (set of tuples): Set of (x, y) positions occupied by clay.
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
        min_y (int): Minimum y-coordinate to display.
        max_y (int): Maximum y-coordinate to display.
    """
    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = 0  # Starting at y=0
    grid_max_y = max_y

    # Print x-axis labels
    print_x_labels(grid_min_x, grid_max_x)

    # Render each row of the grid
    for y in range(grid_min_y, grid_max_y + 1):
        line = ""
        for x in range(grid_min_x, grid_max_x + 1):
            if (x, y) == (500, 0):
                line += '+'  # Water spring
            elif (x, y) in clay:
                line += '#'  # Clay
            else:
                line += '.'  # Sand
        print(f"{y:>2} {line}")  # Right-align y for better formatting

def main():
    # Example input representing clay veins
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    # Parse the input to extract clay positions and grid boundaries
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Visualize the grid
    visualize(clay, min_x, max_x, min_y, max_y)

if __name__ == "__main__":
    main()

   44444455555555
   99999900000000
   45678901234567
 0 ......+.......
 1 ............#.
 2 .#..#.......#.
 3 .#..#..#......
 4 .#..#..#......
 5 .#.....#......
 6 .#.....#......
 7 .#######......
 8 ..............
 9 ..............
10 ....#.....#...
11 ....#.....#...
12 ....#.....#...
13 ....#######...


In [ ]:
import re
from collections import deque

def parse_input(input_lines):
    """
    Parses the input lines to extract clay vein positions.

    Args:
        input_lines (list of str): Each string represents a clay vein definition.

    Returns:
        tuple: A tuple containing:
            - clay (set of tuples): Set of (x, y) positions occupied by clay.
            - min_x (int): Minimum x-coordinate in the clay veins.
            - max_x (int): Maximum x-coordinate in the clay veins.
            - min_y (int): Minimum y-coordinate in the clay veins.
            - max_y (int): Maximum y-coordinate in the clay veins.
    """
    clay = set()
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line formats:
        # "x=495, y=2..7"
        # "y=7, x=495..501"
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue  # Skip lines that don't match the expected format
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_x, max_x, min_y, max_y

def print_x_labels(min_x, max_x):
    """
    Prints the x-axis labels above the grid for reference.

    Args:
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
    """
    # Create the three label lines: hundreds, tens, units
    hundreds = ""
    tens = ""
    units = ""
    for x in range(min_x, max_x + 1):
        hundreds += str(x // 100)
    for x in range(min_x, max_x + 1):
        tens += str((x % 100) // 10)
    for x in range(min_x, max_x + 1):
        units += str(x % 10)
    print("     " + hundreds)
    print("     " + tens)
    print("     " + units)

def visualize(grid, min_x, max_x, min_y, max_y):
    """
    Renders the grid with clay, sand, water flowing, settled water, and the water spring.

    Args:
        grid (dict): Dictionary mapping (x, y) to symbols ('#', '.', '|', '~', '+').
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
        min_y (int): Minimum y-coordinate to display.
        max_y (int): Maximum y-coordinate to display.
    """
    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = min_y
    grid_max_y = max_y

    # Print x-axis labels
    print_x_labels(grid_min_x, grid_max_x)

    # Render each row of the grid
    for y in range(grid_min_y, grid_max_y + 1):
        line = ""
        for x in range(grid_min_x, grid_max_x + 1):
            if (x, y) == (500, 0):
                line += '+'  # Water spring
            elif (x, y) in grid:
                line += grid[(x, y)]
            else:
                line += '.'  # Sand
        print(f"{y:>3} {line}")  # Right-align y for better formatting

def simulate_water(clay, min_x, max_x, min_y, max_y):
    """
    Simulates the water flow based on the clay veins.

    Args:
        clay (set of tuples): Set of (x, y) positions occupied by clay.
        min_x (int): Minimum x-coordinate in the clay veins.
        max_x (int): Maximum x-coordinate in the clay veins.
        min_y (int): Minimum y-coordinate in the clay veins.
        max_y (int): Maximum y-coordinate in the clay veins.

    Returns:
        dict: Grid mapping (x, y) to symbols ('#', '.', '|', '~', '+').
    """
    grid = {}
    for (x, y) in clay:
        grid[(x, y)] = '#'  # Clay

    stack = deque()
    stack.append((500, 0))  # Water spring

    while stack:
        x, y = stack.pop()

        # If y is below the scanning range, ignore
        if y > max_y:
            continue

        # If the position is already settled, no need to process
        if grid.get((x, y)) == '~':
            continue

        # Move downward
        below = (x, y + 1)
        if below not in grid:
            grid[(x, y)] = '|'  # Mark as flowing
            stack.append((x, y + 1))
            continue
        elif grid.get(below) in ['|']:
            grid[(x, y)] = '|'  # Still flowing
            continue
        elif grid.get(below) in ['#', '~']:
            # Attempt to spread left and right
            left_bound = spread(x, y, -1, grid, clay, max_y)
            right_bound = spread(x, y, 1, grid, clay, max_y)

            # Check if water is bounded on both sides
            if left_bound and right_bound:
                # Settle water
                for spread_x in range(left_bound[0] + 1, right_bound[0]):
                    grid[(spread_x, y)] = '~'
                # After settling, revisit the row above
                stack.append((x, y - 1))
            else:
                # Water flows out; mark boundaries as flowing
                for spread_x in range(left_bound[0] + 1, right_bound[0]):
                    if grid.get((spread_x, y)) != '#':
                        grid[(spread_x, y)] = '|'

    return grid

def spread(x, y, direction, grid, clay, max_y):
    """
    Spreads water to the left or right from the current position.

    Args:
        x (int): Current x-coordinate.
        y (int): Current y-coordinate.
        direction (int): -1 for left, 1 for right.
        grid (dict): Current grid state.
        clay (set of tuples): Set of clay positions.
        max_y (int): Maximum y-coordinate in the grid.

    Returns:
        tuple: (x, y) position where spreading stopped.
    """
    spread_x = x
    while True:
        spread_x += direction
        current = (spread_x, y)
        below = (spread_x, y + 1)

        if current in clay:
            return (spread_x, y)  # Bounded by clay

        if below not in clay and grid.get(below) != '~':
            # Water can fall down; mark as flowing and stop spreading
            grid[current] = '|'
            return (spread_x, y)

        # Mark as flowing
        grid[current] = '|'

def count_reachable_tiles(grid, min_y, max_y):
    """
    Counts the number of tiles that water can reach within the specified y-range.

    Args:
        grid (dict): Grid mapping (x, y) to symbols ('#', '.', '|', '~', '+').
        min_y (int): Minimum y-coordinate to consider.
        max_y (int): Maximum y-coordinate to consider.

    Returns:
        int: Number of reachable tiles.
    """
    count = 0
    for (x, y), val in grid.items():
        if min_y <= y <= max_y and val in ['|', '~']:
            count += 1
    return count

def main():
    # Example input representing clay veins
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    # Parse the input to extract clay positions and grid boundaries
    clay, min_x, max_x, min_y, max_y = parse_input(input_data)

    # Simulate water flow
    grid = simulate_water(clay, min_x, max_x, min_y, max_y)

    # Count reachable tiles
    reachable = count_reachable_tiles(grid, min_y, max_y)
    print(f"Number of tiles the water can reach: {reachable}\n")

    # Visualize the final state
    visualize(grid, min_x, max_x, min_y, max_y)

if __name__ == "__main__":
    main()

In [4]:
import re
from collections import deque

def parse_input(input_lines):
    """
    Parses the input lines to extract clay vein positions.

    Args:
        input_lines (list of str): Each string represents a clay vein definition.

    Returns:
        tuple: A tuple containing:
            - clay (set of tuples): Set of (x, y) positions occupied by clay.
            - min_x (int): Minimum x-coordinate in the clay veins.
            - max_x (int): Maximum x-coordinate in the clay veins.
            - min_y (int): Minimum y-coordinate in the clay veins.
            - max_y (int): Maximum y-coordinate in the clay veins.
    """
    clay = set()
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    for line in input_lines:
        # Example line formats:
        # "x=495, y=2..7"
        # "y=7, x=495..501"
        m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
        if not m:
            continue  # Skip lines that don't match the expected format
        fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
        fixed_val = int(fixed_val)
        var_start = int(var_start)
        var_end = int(var_end)
        for var in range(var_start, var_end + 1):
            if fixed_axis == 'x':
                x = fixed_val
                y = var
            else:
                y = fixed_val
                x = var
            clay.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    return clay, min_x, max_x, min_y, max_y

def print_x_labels(min_x, max_x):
    """
    Prints the x-axis labels above the grid for reference.

    Args:
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
    """
    # Create the three label lines: hundreds, tens, units
    hundreds = ""
    tens = ""
    units = ""
    for x in range(min_x, max_x + 1):
        hundreds += str(x // 100)
    for x in range(min_x, max_x + 1):
        tens += str((x % 100) // 10)
    for x in range(min_x, max_x + 1):
        units += str(x % 10)
    print("     " + hundreds)
    print("     " + tens)
    print("     " + units)

def visualize(grid, min_x, max_x, min_y, max_y):
    """
    Renders the grid with clay, sand, water flowing, settled water, and the water spring.

    Args:
        grid (dict): Dictionary mapping (x, y) to symbols ('#', '.', '|', '~', '+').
        min_x (int): Minimum x-coordinate in the grid.
        max_x (int): Maximum x-coordinate in the grid.
        min_y (int): Minimum y-coordinate to display.
        max_y (int): Maximum y-coordinate to display.
    """
    # Adjust the grid to include some padding
    padding = 1
    grid_min_x = min_x - padding
    grid_max_x = max_x + padding
    grid_min_y = min_y
    grid_max_y = max_y

    # Print x-axis labels
    print_x_labels(grid_min_x, grid_max_x)

    # Render each row of the grid
    for y in range(grid_min_y, grid_max_y + 1):
        line = ""
        for x in range(grid_min_x, grid_max_x + 1):
            if (x, y) == (500, 0):
                line += '+'  # Water spring
            elif (x, y) in grid:
                line += grid[(x, y)]
            else:
                line += '.'  # Sand
        print(f"{y:>3} {line}")  # Right-align y for better formatting

def simulate_water_flow(input_lines, water_units):
    """
    Simulates the water flow based on clay veins and a specified number of water units.

    Args:
        input_lines (list of str): Each string represents a clay vein definition.
        water_units (int): Number of water units to simulate.

    Returns:
        None
    """
    # Parse the input to extract clay positions and grid boundaries
    clay, min_x, max_x, min_y, max_y = parse_input(input_lines)

    # Initialize the grid with clay positions
    grid = {}
    for (x, y) in clay:
        grid[(x, y)] = '#'  # Clay

    # Define the spring position
    spring = (500, 0)
    grid[spring] = '+'  # Water spring

    # Function to simulate one water unit
    def simulate_one_unit():
        """
        Simulates the path of a single water unit from the spring.

        Returns:
            bool: True if the water unit settled within the grid, False if it flowed out.
        """
        stack = deque()
        stack.append(spring)

        while stack:
            x, y = stack.pop()

            # Move downward as long as possible
            while True:
                below = (x, y + 1)
                if y + 1 > max_y:
                    return False  # Water flowed out
                if below not in grid:
                    grid[below] = '|'  # Flowing water
                    y += 1
                elif grid[below] in ['|']:
                    # Water is flowing; cannot settle
                    return False
                elif grid[below] in ['#', '~']:
                    break  # Blocked; need to spread

            # Attempt to spread left and right
            left_bound = x
            right_bound = x
            can_settle = True

            # Spread to the left
            while True:
                left_bound -= 1
                current = (left_bound, y)
                below = (left_bound, y + 1)
                if current in clay:
                    left_bound += 1
                    break
                if below not in grid or grid.get(below) in ['|']:
                    # Water can fall down here
                    can_settle = False
                    stack.append((left_bound, y))
                    break
                grid[current] = '|'  # Mark as flowing

            # Spread to the right
            while True:
                right_bound += 1
                current = (right_bound, y)
                below = (right_bound, y + 1)
                if current in clay:
                    right_bound -= 1
                    break
                if below not in grid or grid.get(below) in ['|']:
                    # Water can fall down here
                    can_settle = False
                    stack.append((right_bound, y))
                    break
                grid[current] = '|'  # Mark as flowing

            if can_settle:
                # Water is bounded; settle it
                for fill_x in range(left_bound, right_bound + 1):
                    grid[(fill_x, y)] = '~'
                # After settling, the water above might also settle
                stack.append((x, y - 1))

            return True  # Water unit settled

    # Simulate the specified number of water units
    settled_units = 0
    for _ in range(water_units):
        settled = simulate_one_unit()
        if settled:
            settled_units += 1
        else:
            # Water flowed out; no longer count as settled
            pass

    # Count reachable tiles (flowing and settled water) within y-range
    reachable = 0
    for (x, y), val in grid.items():
        if min_y <= y <= max_y and val in ['|', '~']:
            reachable += 1

    # Print the number of reachable tiles
    print(f"Number of tiles the water can reach: {reachable}\n")

    # Visualize the final state
    visualize(grid, min_x, max_x, min_y, max_y)

# Example usage
if __name__ == "__main__":
    # Example input representing clay veins
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    # Number of water units to simulate
    water_units = 5

    # Simulate water flow and visualize the state after the specified number of units
    simulate_water_flow(input_data, water_units)

Number of tiles the water can reach: 10

     44444455555555
     99999900000000
     45678901234567
  1 ......|.....#.
  2 .#..#.|.....#.
  3 .#..#.|#......
  4 .#..#.|#......
  5 .#....|#......
  6 .#~~~~~#......
  7 .#######......
  8 ..............
  9 ..............
 10 ....#.....#...
 11 ....#.....#...
 12 ....#.....#...
 13 ....#######...


In [1]:
print(1)

1


In [10]:
import re
from collections import deque
print(1)
def simulate_water_flow(input_string, water_units):
    """
    Simulates water flow based on clay veins and visualizes the grid state after a specified number of water units.

    Args:
        input_string (str): Multi-line string representing clay vein definitions.
        water_units (int): Number of water units to simulate.

    Returns:
        None: Prints the number of reachable tiles and the final grid visualization.
    """
    
    def parse_input(input_lines):
        """
        Parses the input lines to extract clay vein positions.

        Args:
            input_lines (list of str): Each string represents a clay vein definition.

        Returns:
            tuple: A tuple containing:
                - clay (set of tuples): Set of (x, y) positions occupied by clay.
                - min_x (int): Minimum x-coordinate in the clay veins.
                - max_x (int): Maximum x-coordinate in the clay veins.
                - min_y (int): Minimum y-coordinate in the clay veins.
                - max_y (int): Maximum y-coordinate in the clay veins.
        """
        clay = set()
        min_x = float('inf')
        max_x = float('-inf')
        min_y = float('inf')
        max_y = float('-inf')

        for line in input_lines:
            # Example line formats:
            # "x=495, y=2..7"
            # "y=7, x=495..501"
            m = re.match(r'(x|y)=(\d+), (x|y)=(\d+)..(\d+)', line)
            if not m:
                continue  # Skip lines that don't match the expected format
            fixed_axis, fixed_val, var_axis, var_start, var_end = m.groups()
            fixed_val = int(fixed_val)
            var_start = int(var_start)
            var_end = int(var_end)
            for var in range(var_start, var_end + 1):
                if fixed_axis == 'x':
                    x = fixed_val
                    y = var
                else:
                    y = fixed_val
                    x = var
                clay.add((x, y))
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)
        return clay, min_x, max_x, min_y, max_y

    def print_x_labels(min_x, max_x):
        """
        Prints the x-axis labels above the grid for reference.

        Args:
            min_x (int): Minimum x-coordinate in the grid.
            max_x (int): Maximum x-coordinate in the grid.
        """
        # Create the three label lines: hundreds, tens, units
        hundreds = ""
        tens = ""
        units = ""
        for x in range(min_x, max_x + 1):
            hundreds += str(x // 100)
        for x in range(min_x, max_x + 1):
            tens += str((x % 100) // 10)
        for x in range(min_x, max_x + 1):
            units += str(x % 10)
        print("     " + hundreds)
        print("     " + tens)
        print("     " + units)

    def visualize(grid, min_x, max_x, min_y, max_y):
        """
        Renders the grid with clay, sand, flowing water, settled water, and the water spring.

        Args:
            grid (dict): Dictionary mapping (x, y) to symbols ('#', '.', '|', '~', '+').
            min_x (int): Minimum x-coordinate in the grid.
            max_x (int): Maximum x-coordinate in the grid.
            min_y (int): Minimum y-coordinate to display.
            max_y (int): Maximum y-coordinate to display.
        """
        # Adjust the grid to include some padding
        padding = 1
        grid_min_x = min_x - padding
        grid_max_x = max_x + padding
        grid_min_y = min_y
        grid_max_y = max_y

        # Print x-axis labels
        print_x_labels(grid_min_x, grid_max_x)

        # Render each row of the grid
        for y in range(grid_min_y, grid_max_y + 1):
            line = ""
            for x in range(grid_min_x, grid_max_x + 1):
                if (x, y) == (500, 0):
                    line += '+'  # Water spring
                elif (x, y) in grid:
                    line += grid[(x, y)]
                else:
                    line += '.'  # Sand
            print(f"{y:>3} {line}")  # Right-align y for better formatting

    def simulate_water_flow_core(clay, min_x, max_x, min_y, max_y, water_units):
        """
        Core simulation function that injects water units and updates the grid.

        Args:
            clay (set of tuples): Set of (x, y) positions occupied by clay.
            min_x (int): Minimum x-coordinate in the clay veins.
            max_x (int): Maximum x-coordinate in the clay veins.
            min_y (int): Minimum y-coordinate in the clay veins.
            max_y (int): Maximum y-coordinate in the clay veins.
            water_units (int): Number of water units to simulate.

        Returns:
            dict: Updated grid after simulation.
        """
        grid = {}
        for (x, y) in clay:
            grid[(x, y)] = '#'  # Clay

        # Define the spring position
        spring = (500, 0)
        grid[spring] = '+'  # Water spring

        # Directions for spreading: left (-1) and right (+1)
        directions = [-1, 1]

        for unit in range(water_units):
            stack = deque()
            stack.append(spring)

            while stack:
                x, y = stack.pop()

                # Move downward as far as possible
                while True:
                    below = (x, y + 1)
                    if y + 1 > max_y:
                        # Water flows out of the grid
                        break
                    if below not in grid:
                        grid[below] = '|'  # Mark as flowing
                        y += 1
                    elif grid.get(below) in ['|']:
                        # Water is flowing below; cannot settle
                        break
                    elif grid.get(below) in ['#', '~']:
                        # Blocked; need to spread left and right
                        break

                # After moving down, attempt to spread left and right
                if y + 1 > max_y:
                    # Water has flowed out; do not mark current position
                    continue

                # Attempt to spread left and right
                spread_results = {}
                for direction in directions:
                    spread_x = x
                    spread_y = y
                    while True:
                        spread_x += direction
                        current = (spread_x, spread_y)
                        below = (spread_x, spread_y + 1)

                        if current in clay:
                            # Bounded by clay on this side
                            spread_results[direction] = spread_x
                            break
                        if below not in grid or grid.get(below) in ['|']:
                            # Water can fall down here; mark as flowing and stop
                            grid[current] = '|'
                            stack.append(current)
                            spread_results[direction] = spread_x
                            break
                        else:
                            # Mark as flowing
                            grid[current] = '|'

                # Determine if water is bounded on both sides
                left_bound = spread_results.get(-1, x)
                right_bound = spread_results.get(1, x)

                if spread_results.get(-1, None) != x and spread_results.get(1, None) != x:
                    # Both sides are bounded by clay
                    for fill_x in range(left_bound, right_bound + 1):
                        grid[(fill_x, y)] = '~'
                    # After settling, need to revisit the row above
                    stack.append((x, y - 1))
                else:
                    # Water flows out; do not settle
                    pass

        return grid

    # Parsing input and handling visualization
    def main_simulation(input_string, water_units):
        """
        Orchestrates the simulation and visualization.

        Args:
            input_string (str): Multi-line string representing clay vein definitions.
            water_units (int): Number of water units to simulate.

        Returns:
            None: Prints the number of reachable tiles and the final grid visualization.
        """
        # Split the input string into lines
        input_lines = input_string.strip().split('\n')
        
        # Parse the input to extract clay positions and grid boundaries
        clay, min_x, max_x, min_y, max_y = parse_input(input_lines)

        # Simulate water flow
        grid = simulate_water_flow_core(clay, min_x, max_x, min_y, max_y, water_units)
        print(2)
        # Count reachable tiles (flowing and settled water) within y-range
        reachable = 0
        for (x, y), val in grid.items():
            if min_y <= y <= max_y and val in ['|', '~']:
                reachable += 1
        print(2)
        # Print the number of reachable tiles
        print(f"Number of tiles the water can reach: {reachable}\n")

        # Visualize the final state
        visualize(grid, min_x, max_x, min_y, max_y)

    # Example usage with different water units
    if __name__ == "__main__":
        # Example input representing clay veins
        input_string = """
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504
"""

        # Define different water units for testing
        test_units = [1, 4, 5, 9, 10]
        print(2)
        for units in test_units:
            print(f"\n--- Simulation after injecting {units} water unit(s) ---\n")
            main_simulation(input_string, units)


1


In [12]:
def simulate_water_flow(input_lines, water_units):
    """
    Simulates the water flow based on clay veins and a specified number of water units.

    Args:
        input_lines (list of str): Each string represents a clay vein definition.
        water_units (int): Number of water units to simulate.

    Returns:
        None
    """
    clay, min_x, max_x, min_y, max_y = parse_input(input_lines)

    # Initialize the grid with clay positions
    grid = {}
    for (x, y) in clay:
        grid[(x, y)] = '#'  # Clay

    # Define the spring position
    spring = (500, 0)
    grid[spring] = '+'  # Water spring

    def simulate_one_unit():
        """
        Simulates the path of a single water unit from the spring.
        Returns:
            bool: True if the water unit settled, False if it flowed out.
        """
        stack = deque([spring])
        current_flow = set()  # Tracks the flowing path of this water unit

        while stack:
            x, y = stack.pop()

            # Move downward if possible
            while True:
                below = (x, y + 1)
                if y + 1 > max_y:
                    # Water flowed out of bounds
                    return False
                if below not in grid:
                    grid[below] = '|'  # Mark as flowing water
                    current_flow.add(below)
                    y += 1
                elif grid[below] in ['|']:
                    # Cannot settle, as below is flowing
                    return False
                elif grid[below] in ['#', '~']:
                    break  # Blocked below; spread left and right

            # Spread left and right
            left_bound = x
            right_bound = x
            can_settle = True

            # Spread left
            while True:
                left_bound -= 1
                current = (left_bound, y)
                below = (left_bound, y + 1)
                if current in clay:
                    left_bound += 1
                    break
                if below not in grid or grid.get(below) in ['|']:
                    can_settle = False
                    stack.append((left_bound, y))
                    break
                grid[current] = '|'  # Flowing water
                current_flow.add(current)

            # Spread right
            while True:
                right_bound += 1
                current = (right_bound, y)
                below = (right_bound, y + 1)
                if current in clay:
                    right_bound -= 1
                    break
                if below not in grid or grid.get(below) in ['|']:
                    can_settle = False
                    stack.append((right_bound, y))
                    break
                grid[current] = '|'  # Flowing water
                current_flow.add(current)

            if can_settle:
                # Bounded region; water settles
                for fill_x in range(left_bound, right_bound + 1):
                    grid[(fill_x, y)] = '~'
                return True

        return False  # No settlement possible

    # Simulate water units
    for unit in range(1, water_units + 1):
        simulate_one_unit()
        print(f"\nAfter {unit} units of water:\n")
        visualize(grid, min_x, max_x, min_y, max_y)
if __name__ == "__main__":
    input_data = [
        "x=495, y=2..7",
        "y=7, x=495..501",
        "x=501, y=3..7",
        "x=498, y=2..4",
        "x=506, y=1..2",
        "x=498, y=10..13",
        "x=504, y=10..13",
        "y=13, x=498..504"
    ]

    water_units = 3  # Simulate 3 units of water
    simulate_water_flow(input_data, water_units)



UnboundLocalError: cannot access local variable 'parse_input' where it is not associated with a value

In [3]:
import re
from collections import defaultdict

# Read the input file
with open('input.txt', 'r') as file:
    data = file.readlines()

# Parse the clay positions
clay_positions = set()
for line in data:
    match = re.match(r"(x|y)=(\d+), (x|y)=(\d+)\.\.(\d+)", line.strip())
    if match:
        fixed_axis, fixed_value, range_axis, range_start, range_end = match.groups()
        fixed_value = int(fixed_value)
        range_start = int(range_start)
        range_end = int(range_end)
        if fixed_axis == "x":
            for y in range(range_start, range_end + 1):
                clay_positions.add((fixed_value, y))
        else:
            for x in range(range_start, range_end + 1):
                clay_positions.add((x, fixed_value))

# Define the grid boundaries
min_x = min(x for x, _ in clay_positions) - 1
max_x = max(x for x, _ in clay_positions) + 1
min_y = min(y for _, y in clay_positions)
max_y = max(y for _, y in clay_positions)

# Water flow simulation
def flow_water(x, y, grid, visited):
    if (x, y) in visited or y > max_y:
        return
    visited.add((x, y))
    grid[y][x] = '|'
    
    # Flow down
    if (x, y + 1) not in clay_positions and grid[y + 1][x] != '~':
        flow_water(x, y + 1, grid, visited)
    
    # Spread left and right if below is blocked or settled
    if (x, y + 1) in clay_positions or grid[y + 1][x] == '~':
        left_bound, right_bound = x, x
        while (left_bound - 1, y) not in clay_positions and (left_bound - 1, y + 1) in clay_positions:
            left_bound -= 1
            grid[y][left_bound] = '|'
            visited.add((left_bound, y))
        while (right_bound + 1, y) not in clay_positions and (right_bound + 1, y + 1) in clay_positions:
            right_bound += 1
            grid[y][right_bound] = '|'
            visited.add((right_bound, y))
        # Check if it's closed off to fill
        if (left_bound - 1, y) in clay_positions and (right_bound + 1, y) in clay_positions:
            for x_fill in range(left_bound, right_bound + 1):
                grid[y][x_fill] = '~'

# Create a grid to simulate
grid = defaultdict(lambda: defaultdict(lambda: '.'))
for x, y in clay_positions:
    grid[y][x] = '#'

# Simulate water flow
spring_x, spring_y = 500, 0
visited = set()
flow_water(spring_x, spring_y, grid, visited)

# Count reachable tiles
reachable_tiles = sum(1 for (x, y) in visited if min_y <= y <= max_y)
reachable_tiles

38

In [2]:
from collections import deque

SPRING_LOCATION = (500, 0)
SPRING = 0
SAND = 1
DRY = 2
WATER = 3
CLAY = 4

def parse_input(file):
    with open(file, "r") as f:
        input_data = f.read()
    
    clay_coordinates = []
    max_x, max_y = 0, 0

    for line in input_data.strip().split("\n"):
        parts = line.split(", ")
        a, b = parts[0], parts[1]
        a_parts = a.split("=")
        b_parts = b.split("=")
        a_val = int(a_parts[1])
        b_range = list(map(int, b_parts[1].split("..")))

        if a_parts[0] == "x":
            for y in range(b_range[0], b_range[1] + 1):
                clay_coordinates.append((a_val, y))
                max_x = max(max_x, a_val)
                max_y = max(max_y, y)
        else:
            for x in range(b_range[0], b_range[1] + 1):
                clay_coordinates.append((x, a_val))
                max_x = max(max_x, x)
                max_y = max(max_y, a_val)
    
    grid = [[SAND for _ in range(max_y + 1)] for _ in range(max_x + 2)]
    for x, y in clay_coordinates:
        grid[x][y] = CLAY

    grid[SPRING_LOCATION[0]][SPRING_LOCATION[1]] = SPRING
    return grid, (0, max_y)

def print_grid(grid):
    for row in grid:
        for cell in row:
            symbol = {
                SAND: '.',
                CLAY: '#',
                SPRING: '+',
                WATER: '~',
                DRY: '|'
            }.get(cell, ' ')
            print(symbol, end="")
        print()

def simulate_water(grid, area):
    springs = deque([SPRING_LOCATION])

    while springs:
        x, y = springs.popleft()
        ny = y + 1
        while ny < len(grid[0]) and grid[x][ny] == SAND:
            grid[x][ny] = DRY
            ny += 1

        if ny < len(grid[0]) and (grid[x][ny] == CLAY or grid[x][ny] == WATER):
            left, right = x, x

            while left > 0 and grid[left - 1][ny - 1] != CLAY and grid[left][ny] != DRY:
                left -= 1

            while right < len(grid) - 1 and grid[right + 1][ny - 1] != CLAY and grid[right][ny] != DRY:
                right += 1

            below_filled = all(grid[cx][ny] in (CLAY, WATER) for cx in range(left, right + 1))
            if below_filled:
                for cx in range(left, right + 1):
                    grid[cx][ny - 1] = WATER
            else:
                for cx in range(left, right + 1):
                    if grid[cx][ny - 1] != CLAY:
                        grid[cx][ny - 1] = DRY
                        if grid[cx][ny] == SAND:
                            springs.append((cx, ny - 1))

    dry_cells = sum(row[area[0]:area[1] + 1].count(DRY) for row in grid)
    water_cells = sum(row[area[0]:area[1] + 1].count(WATER) for row in grid)

    return dry_cells + water_cells, water_cells

if __name__ == "__main__":
    grid, area = parse_input("input.txt")
    result = simulate_water(grid, area)
    # print_grid(grid)
    print(f"Result: {result}")


Result: (42, 20)


In [4]:
# Reinitialize grid and visited set for the recursive solution
grid = defaultdict(lambda: defaultdict(lambda: '.'))
for x, y in clay_positions:
    grid[y][x] = '#'

visited = set()

# Recursive water flow function with proper filling and overflow logic
def flow_recursive(x, y):
    if y > max_y or (x, y) in visited:
        return
    visited.add((x, y))
    
    # Flow down if possible
    if grid[y + 1][x] not in ('#', '~'):
        grid[y][x] = '|'
        flow_recursive(x, y + 1)
    # Spread left and right if blocked below or settled
    if grid[y + 1][x] in ('#', '~'):
        left_bound, right_bound = x, x
        can_fill = True

        # Spread left
        while (left_bound - 1, y) not in clay_positions and grid[y + 1][left_bound - 1] in ('#', '~'):
            left_bound -= 1
            grid[y][left_bound] = '|'
            visited.add((left_bound, y))
        if (left_bound - 1, y) not in clay_positions:
            can_fill = False

        # Spread right
        while (right_bound + 1, y) not in clay_positions and grid[y + 1][right_bound + 1] in ('#', '~'):
            right_bound += 1
            grid[y][right_bound] = '|'
            visited.add((right_bound, y))
        if (right_bound + 1, y) not in clay_positions:
            can_fill = False

        # If fully enclosed, fill with water
        if can_fill:
            for x_fill in range(left_bound, right_bound + 1):
                grid[y][x_fill] = '~'
            flow_recursive(x, y - 1)  # Flow upward to potentially fill more

# Start the recursive water flow from the spring
flow_recursive(spring_x, spring_y)

# Count the total reachable tiles
reachable_water_tiles = sum(
    1 for y in range(min_y, max_y + 1)
    for x in range(min_x - 1, max_x + 2)
    if grid[y][x] in ('|', '~')
)
reachable_water_tiles


267